In [2]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, concatenate, Convolution1D
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from keras.callbacks import EarlyStopping
import codecs

import io
import random as rn
import tensorflow as tf

Using TensorFlow backend.


In [3]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [4]:

# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

# text feature
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
#vad = np.load('../../data/y_egemaps.npy')
# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))
#g_word_embedding_matrix = np.load(path+'g_word_embedding_matrix.npy')

# other parameters
# MAX_SEQUENCE_LENGTH = 554
EMBEDDING_DIM = 300
# nb_words = 3438

Found 3437 unique tokens


In [5]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [6]:
# split train/test
split = 7869

In [7]:
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
                    #weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    #net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
#     net = Bidirectional(LSTM(64, return_sequences=True))(net)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    #out_text = Dropout(0.5)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model
model = text_model1()
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     1031400     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 256)     439296      embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 128)          164352      bidirectional_1[0][0]            
___________

In [8]:
# main function for for LOSO
def main(time):
    model = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_text[:split], 
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model.evaluate( x_train_text[split:], vad[split:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    weight = []
    for time in range(30):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('text_blstm_WE7869_no_dropout.csv', header=['time','v','a','d', 'ave'])



Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 49s 8ms/step - loss: 1.8303 - v_loss: 0.5895 - a_loss: 0.6303 - d_loss: 0.6104 - v_ccc: 0.4101 - a_ccc: 0.3697 - d_ccc: 0.3899 - val_loss: 1.9613 - val_v_loss: 0.6973 - val_a_loss: 0.6534 - val_d_loss: 0.6350 - val_v_ccc: 0.3090 - val_a_ccc: 0.3551 - val_d_ccc: 0.3747
Epoch 2/50
6295/6295 [==============================] - 47s 7ms/step - loss: 1.4001 - v_loss: 0.3807 - a_loss: 0.5165 - d_loss: 0.5029 - v_ccc: 0.6194 - a_ccc: 0.4833 - d_ccc: 0.4973 - val_loss: 1.9109 - val_v_loss: 0.6579 - val_a_loss: 0.6268 - val_d_loss: 0.6491 - val_v_ccc: 0.3468 - val_a_ccc: 0.3818 - val_d_ccc: 0.3605
Epoch 3/50
6295/6295 [==============================] - 45s 7ms/step - loss: 1.2474 - v_loss: 0.3289 - a_loss: 0.4656 - d_loss: 0.4529 - v_ccc: 0.6712 - a_ccc: 0.5342 - d_ccc: 0.5473 - val_loss: 1.9219 - val_v_loss: 0.6154 - val_a_loss: 0.6310 - val_d_loss: 0.6978 - val_v_ccc: 0.3912 - val_a_ccc: 0.3

Epoch 9/50
6295/6295 [==============================] - 47s 7ms/step - loss: 0.7895 - v_loss: 0.1996 - a_loss: 0.2969 - d_loss: 0.2930 - v_ccc: 0.8004 - a_ccc: 0.7032 - d_ccc: 0.7069 - val_loss: 1.8911 - val_v_loss: 0.5873 - val_a_loss: 0.6387 - val_d_loss: 0.6842 - val_v_ccc: 0.4180 - val_a_ccc: 0.3672 - val_d_ccc: 0.3237
Epoch 10/50
6295/6295 [==============================] - 47s 7ms/step - loss: 0.7542 - v_loss: 0.1912 - a_loss: 0.2819 - d_loss: 0.2808 - v_ccc: 0.8088 - a_ccc: 0.7180 - d_ccc: 0.7190 - val_loss: 1.9716 - val_v_loss: 0.5732 - val_a_loss: 0.7053 - val_d_loss: 0.7130 - val_v_ccc: 0.4323 - val_a_ccc: 0.3004 - val_d_ccc: 0.2957
Epoch 11/50
6295/6295 [==============================] - 47s 7ms/step - loss: 0.7136 - v_loss: 0.1769 - a_loss: 0.2706 - d_loss: 0.2668 - v_ccc: 0.8233 - a_ccc: 0.7297 - d_ccc: 0.7334 - val_loss: 1.8937 - val_v_loss: 0.5703 - val_a_loss: 0.6608 - val_d_loss: 0.6834 - val_v_ccc: 0.4362 - val_a_ccc: 0.3457 - val_d_ccc: 0.3244
Epoch 12/50
6295/6295 [

Epoch 11/50
6295/6295 [==============================] - 48s 8ms/step - loss: 0.7139 - v_loss: 0.1754 - a_loss: 0.2717 - d_loss: 0.2663 - v_ccc: 0.8244 - a_ccc: 0.7281 - d_ccc: 0.7335 - val_loss: 1.9291 - val_v_loss: 0.5688 - val_a_loss: 0.6751 - val_d_loss: 0.7056 - val_v_ccc: 0.4384 - val_a_ccc: 0.3302 - val_d_ccc: 0.3023
Epoch 12/50
6295/6295 [==============================] - 48s 8ms/step - loss: 0.6708 - v_loss: 0.1649 - a_loss: 0.2534 - d_loss: 0.2523 - v_ccc: 0.8351 - a_ccc: 0.7465 - d_ccc: 0.7476 - val_loss: 1.9181 - val_v_loss: 0.5804 - val_a_loss: 0.6732 - val_d_loss: 0.6882 - val_v_ccc: 0.4270 - val_a_ccc: 0.3326 - val_d_ccc: 0.3223
Epoch 13/50
2170/2170 [==============================] - 4s 2ms/step
[0.3549552857875824, 0.40131253004074097, 0.3984972834587097]
2
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 50s 8ms/step - loss: 1.8417 - v_loss: 0.6076 - a_loss: 0.6251 - d_loss: 0.6087 - v_ccc: 0.3923 - a_ccc: 0.3749 

6295/6295 [==============================] - 47s 7ms/step - loss: 1.2349 - v_loss: 0.3239 - a_loss: 0.4639 - d_loss: 0.4481 - v_ccc: 0.6762 - a_ccc: 0.5369 - d_ccc: 0.5520 - val_loss: 1.9761 - val_v_loss: 0.6222 - val_a_loss: 0.6507 - val_d_loss: 0.7248 - val_v_ccc: 0.3839 - val_a_ccc: 0.3572 - val_d_ccc: 0.2828
Epoch 4/50
6295/6295 [==============================] - 46s 7ms/step - loss: 1.1234 - v_loss: 0.2904 - a_loss: 0.4219 - d_loss: 0.4108 - v_ccc: 0.7096 - a_ccc: 0.5780 - d_ccc: 0.5891 - val_loss: 1.9112 - val_v_loss: 0.6043 - val_a_loss: 0.6421 - val_d_loss: 0.6849 - val_v_ccc: 0.4005 - val_a_ccc: 0.3647 - val_d_ccc: 0.3237
Epoch 5/50
6295/6295 [==============================] - 47s 7ms/step - loss: 1.0367 - v_loss: 0.2633 - a_loss: 0.3875 - d_loss: 0.3857 - v_ccc: 0.7366 - a_ccc: 0.6125 - d_ccc: 0.6141 - val_loss: 1.8698 - val_v_loss: 0.5879 - val_a_loss: 0.6290 - val_d_loss: 0.6778 - val_v_ccc: 0.4181 - val_a_ccc: 0.3795 - val_d_ccc: 0.3326
Epoch 6/50
6295/6295 [==============

Epoch 12/50
6295/6295 [==============================] - 41s 6ms/step - loss: 0.6719 - v_loss: 0.1680 - a_loss: 0.2478 - d_loss: 0.2563 - v_ccc: 0.8320 - a_ccc: 0.7522 - d_ccc: 0.7439 - val_loss: 1.9673 - val_v_loss: 0.5893 - val_a_loss: 0.6871 - val_d_loss: 0.7125 - val_v_ccc: 0.4187 - val_a_ccc: 0.3178 - val_d_ccc: 0.2961
Epoch 13/50
6295/6295 [==============================] - 41s 6ms/step - loss: 0.6422 - v_loss: 0.1596 - a_loss: 0.2453 - d_loss: 0.2371 - v_ccc: 0.8404 - a_ccc: 0.7546 - d_ccc: 0.7628 - val_loss: 1.9948 - val_v_loss: 0.5795 - val_a_loss: 0.7055 - val_d_loss: 0.7313 - val_v_ccc: 0.4299 - val_a_ccc: 0.2989 - val_d_ccc: 0.2764
Epoch 14/50
2170/2170 [==============================] - 4s 2ms/step
[0.3466997444629669, 0.37805187702178955, 0.3689694404602051]
5
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 44s 7ms/step - loss: 1.8383 - v_loss: 0.6034 - a_loss: 0.6190 - d_loss: 0.6150 - v_ccc: 0.3965 - a_ccc: 0.3806 

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6295/6295 [==============================] - 47s 7ms/step - loss: 0.8938 - v_loss: 0.2198 - a_loss: 0.3353 - d_loss: 0.3388 - v_ccc: 0.7802 - a_ccc: 0.6649 - d_ccc: 0.6611 - val_loss: 1.9096 - val_v_loss: 0.5734 - val_a_loss: 0.6700 - val_d_loss: 0.6866 - val_v_ccc: 0.4323 - val_a_ccc: 0.3358 - val_d_ccc: 0.3223
Epoch 8/50
6295/6295 [==============================] - 51s 8ms/step - loss: 0.8368 - v_loss: 0.2094 - a_loss: 0.3141 - d_loss: 0.3134 - v_ccc: 0.7906 - a_ccc: 0.6859 - d_ccc: 0.6867 - val_loss: 1.9207 - val_v_loss: 0.5731 - val_a_loss: 0.6648 - val_d_loss: 0.7027 - val_v_ccc: 0.4333 - val_a_ccc: 0.3409 - val_d_ccc: 0.3051
Epoch 9/50
6295/6295 [==============================] - 51s 8ms/step - loss: 0.7896 - v_loss: 0.1943 - a_loss: 0.2955 - d_loss: 0.3002 - v_ccc: 0.8057 - a_ccc: 0.7047 - d_ccc: 0.7000 - val_loss: 1.9114 - val_v_loss: 0.5539 - val_a_loss: 0.6627 - val_d_loss: 0.7155 - val_v_ccc: 0.4522 - val_a_ccc: 0.3435 - val_d_ccc: 0.2929
Epoch 10/50
6295/6295 [=============

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2170/2170 [==============================] - 5s 2ms/step
[0.37416133284568787, 0.37103453278541565, 0.3828805088996887]
27
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 61s 10ms/step - loss: 1.8623 - v_loss: 0.6080 - a_loss: 0.6286 - d_loss: 0.6260 - v_ccc: 0.3917 - a_ccc: 0.3720 - d_ccc: 0.3740 - val_loss: 2.0328 - val_v_loss: 0.6945 - val_a_loss: 0.6746 - val_d_loss: 0.6901 - val_v_ccc: 0.3141 - val_a_ccc: 0.3340 - val_d_ccc: 0.3191
Epoch 2/50
6295/6295 [==============================] - 52s 8ms/step - loss: 1.4006 - v_loss: 0.3849 - a_loss: 0.5157 - d_loss: 0.5001 - v_ccc: 0.6154 - a_ccc: 0.4842 - d_ccc: 0.4998 - val_loss: 1.9231 - val_v_loss: 0.6181 - val_a_loss: 0.6546 - val_d_loss: 0.6723 - val_v_ccc: 0.3860 - val_a_ccc: 0.3547 - val_d_ccc: 0.3362
Epoch 3/50
6295/6295 [==============================] - 52s 8ms/step - loss: 1.2374 - v_loss: 0.3172 - a_loss: 0.4634 - d_loss: 0.4564 - v_ccc: 0.6828 - a_ccc: 0.5365 - d_ccc: 0.

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

